In [33]:
import os
import pandas as pd
import numpy as np
import settings
pd.set_option('display.max_colwidth', -1)
import json

In [3]:
df = pd.read_csv(os.path.join(settings.DATA_DIR, 'train.csv'))

In [32]:
df[['comment_text', 'target']].sample(10)

,comment_text,target
1048112,"Given you own inability to use punctuation or construct a coherent sentence, Prog, you should be careful responding.",0.000000
1221766,"""Diabetes Canada includes nonnutritive sweeteners in its guidelines, citing Health Canada’s acceptable daily intake values. (The organization has received funding from companies that use nonnutritive sweeteners, though it says no company or representative has any influence in the development of its recommendations.)""\n\nI cannot accept that Diabetes Canada would bite the hand that feeds it, regardless to their claims to the contrary. Are we to not follow the money now?",0.000000
1714643,What kind of rat hole do these monsters come from?,0.644737
92850,"Bravo Shannyn and thanks for another insightful column! Yes, the list of past public safety hypocrisy that has evolved into rational protections for the betterment of our country's future, is endless. Except for one yhuuuuuge exception, common sense gun laws.\n\nWithout any doubt, I know what would turn our country into the most polite on the planet, over night. If all the guns were in the hands of females and none in the hands of males, except for the military and law enforcement. Except for the San Bernardino shooter's wife, all of these mass murders have been perpetrated by insecure males bent on destroying life.",0.300000
1547085,Happy that the Trump Resistance has found another critical issue to get all pumped up about (I see a new Kendzior sermon has been posted).\nThe problem is that many of the schlubs will see these antics as ungrateful and inappropriate slights by spoiled celebrities that target the U.S. in general rather than Trump.,0.000000
230517,and you are stuck in Soldotna...awesome!!,0.000000
1606513,"I was approximating when I said ""April/May"" to keep the comment concise. I believe it was actually part way into June that the red hot start ended, when they reached 20 games over .500. They finished the season 12 games over .500, meaning they were a below-.500 team for the rest of the season after that point (the majority of the season).\n\nYou're right that the whole season still counts, of course. However, when projecting how good a team going forward (like into the playoffs, or into next season), it's prudent to assess the trends. In 2007, for instance, they were also just an average-ish team, but they went on a hot streak at the right time (entering the playoffs). I think this year they were just an average team (a bit below, actually, given that .500 is average) that had an anomalous hot streak at the start of the season. If this same team were fielded next year, I'd project an over/under of about 78 wins.",0.000000
1510581,"Come up with a new smear besides ""dude weed lmao"". People like you are why the republican party no longer holds individual liberty and personal responsibility as core tenets. \n\nIT'S HAPPENING, though. \n\nCome Hell or high water, we will MAKE THE GOP REPUBLICAN AGAIN.",0.200000
973101,"“We must be brave enough to admit the ‘war on terror’ is simply not working,” Corbyn, a longtime peace activist who opposed British involvement in the wars in Iraq and Afghanistan, will say, according to his office. “Many experts, including professionals in our intelligence and security services, have pointed to the connections between wars our government has supported or fought in other countries and terrorism here at home.”\n\nIt's a no brainer, but it goes against monied, and ideological interests which usually Trump sanity. The public will get stuck with more terrorism.",0.000000
567115,"This is so funny how she plays the victim. In another statement she states ""my job is to hold our leaders accountable,"" yet when she is being held accountable she's now the victim. Then to claim she's being punished for speaking at a faux Women's March is dishonest and Rep Theilen should be ashamed of herself for saying otherwise. Anyone who saw and heard that faux Women's March knew it was a anti

In [7]:
AUX_COLUMNS = ['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']
df[AUX_COLUMNS].sample(10)

,target,severe_toxicity,obscene,identity_attack,insult,threat
854812,0.000000,0.0,0.0,0.000000,0.0,0.000000
1165428,0.000000,0.0,0.0,0.000000,0.0,0.000000
1178445,0.100000,0.1,0.0,0.000000,0.0,0.000000
1647547,0.000000,0.0,0.0,0.000000,0.0,0.000000
282432,0.000000,0.0,0.0,0.000000,0.0,0.000000
1346930,0.166667,0.0,0.0,0.166667,0.0,0.166667
195691,0.000000,0.0,0.0,0.000000,0.0,0.000000
1316251,0.000000,0.0,0.0,0.000000,0.0,0.000000
1755413,0.800000,0.0,0.0,0.000000,0.8,0.000000
1021449,0.000000,0.0,0.0,0.000000,0.0,0.000000


In [9]:
print(len(df[df['target']>0.5]), len(df[df['severe_toxicity']>0.5]), len(df[df['obscene']>0.5]),len(df[df['identity_attack']>0.5]),len(df[df['insult']>0.5]), len(df[df['threat']>0.5]))

106438 8 7648 7633 79887 2793


In [10]:
len(df)

1804874

In [26]:
len(df[df['target']==0.5])

37896

In [15]:
len(df[(df['target']>0.5) & (df['insult']>0.5)])

78125

In [16]:
len(df[(df['target']>0.5) & (df['obscene']>0.5)])

7494

In [17]:
df['text_len'] = df.comment_text.map(lambda x: len(x.split(' ')))
len(df[df.text_len > 160])

88410

In [27]:
df_text = pd.read_csv(os.path.join(settings.DATA_DIR, 'test.csv'))
df_text.sample(10)

,id,comment_text
27652,7027652,tryanny doesn't mean what you think it means.
81418,7081418,"Get that you live to troll, Tommy.\n\nNot playing your silly games over the Christmas holidays."
14484,7014484,Except for the Christians in the southern hemisphere....
7792,7007792,NO'Leary !\n\nJust NO !!
83539,7083539,"Washington Post correction: (you can't make this stuff up and make it worse)\n\n\nEDITOR'S NOTE: This story has been updated to more precisely describe White House press secretary Sean Spicer's location late Tuesday night in the minutes before he briefed reporters. Spicer huddled with his staff among bushes near television sets on the White House grounds, not ""in the bushes,"" as the story originally stated.\n\n\nhttps://www.washingtonpost.com/news/post-politics/wp/2017/05/10/as-trump-fired-comey-his-staff-scrambled-to-explain-why/?utm_term=.12f7acda3700"
41306,7041306,"You my friend dream of the Utopia. We all want to live there, unfortunately it does not exist."
4440,7004440,Yes...they've gone crazy this year...good on ya!\n\nWell. Wait until NEXT year (*rolling my eyes*)
45371,7045371,"“Today, the Trump administration took up the cause of oppressed white people.”"
50534,7050534,"Mr. Clark has inadvertently stated the obvious...... ""Suddenly, he has to start thinking ""....\nWas J. Trudeau ""thinking"" when he ordered the removal of our F-18's ? Or was he ""playing to the crowd "". \nRussia has no intention of taking on ""the West"", Putin may be a cruel dictator, but he's not stupid. Russia was made aware of the air-strike at least 8 hours before. As opposed to Canada, one hour before. (yes, Putin was elected)\nOnce again Trudeau has been made irrelevant, and by extension, Canada."
31558,7031558,"Ms Smith: Our current health care system may be the most expensive in the industrialized world; but, at least it's also the least effective. Gary crum"


In [24]:
df_text['text_len'] = df_text.comment_text.map(lambda x: len(x.split(' ')))
len(df_text[df_text.text_len > 200])

62

In [38]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenized input
text = "[CLS] it's http://www.com Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)
tokenized_text

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/core/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


['[CLS]',
 'it',
 "'",
 's',
 'http',
 ':',
 '/',
 '/',
 'www',
 '.',
 'com',
 'who',
 'was',
 'jim',
 'henson',
 '?',
 '[SEP]',
 'jim',
 'henson',
 'was',
 'a',
 'puppet',
 '##eer',
 '[SEP]']